In [1]:
import os
import torch


In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures


In [4]:
dataset=Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
print(f'Dataset: {dataset}:')
print('===============')
print(f'Number of graph: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')




Processing...
Done!


Dataset: Cora():
Number of graph: 1
Number of features: 1433
Number of classes: 7
